# Week 6 - Interactive Visualizations Part 1: Plotly Basics

## Learning Objectives
By the end of this lesson, you will be able to:
1. Create interactive charts using Plotly Express
2. Load and work with real Olist e-commerce data from zip files
3. Build engaging interactive visualizations for business insights
4. Apply hover information and interactive features to enhance data exploration

## Business Context
In today's business environment, static charts are no longer sufficient. Stakeholders want to explore data interactively, drill down into details, and discover insights on their own. Interactive visualizations empower users to:

- **Explore data dynamically** - Zoom, pan, filter in real-time
- **Engage stakeholders** - Interactive elements increase attention and understanding
- **Reveal hidden insights** - Users can discover patterns through exploration
- **Support decision-making** - Interactive dashboards enable data-driven decisions

Today we'll use real Brazilian e-commerce data from Olist to create compelling interactive visualizations that tell business stories.

## 1. Setup and Data Loading

First, let's set up our environment and load real Olist data from the zip file in our repository.

In [1]:
# Import required libraries
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import zipfile
import io
import requests
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Set pandas display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.precision', 2)

print("Libraries loaded successfully!")
print(f"Plotly version: {px.__version__ if hasattr(px, '__version__') else 'Available'}")

Libraries loaded successfully!
Plotly version: Available


In [2]:
def load_olist_data_from_zip():
    """
    Load Olist e-commerce data directly from the GitHub repository zip file.
    This function downloads and extracts real Brazilian e-commerce data.
    
    Returns:
    --------
    dict: Dictionary containing DataFrames for different tables
    """
    try:
        # URL to the sales.zip file in the repository
        zip_url = "https://github.com/autom8or-com/python-data-analysis-course/raw/main/Resources/data/sales.zip"
        
        print("Downloading Olist dataset from repository...")
        response = requests.get(zip_url)
        response.raise_for_status()
        
        # Create a dictionary to store our datasets
        datasets = {}
        
        with zipfile.ZipFile(io.BytesIO(response.content)) as zip_file:
            print("Extracting data files...")
            
            # Load key datasets
            file_mapping = {
                'customers': 'olist_customers_dataset.csv',
                'orders': 'olist_orders_dataset.csv',
                'order_items': 'olist_order_items_dataset.csv',
                'products': 'olist_products_dataset.csv',
                'sellers': 'olist_sellers_dataset.csv',
                'payments': 'olist_order_payments_dataset.csv',
                'reviews': 'olist_order_reviews_dataset.csv',
                'geolocation': 'olist_geolocation_dataset.csv'
            }
            
            for table_name, filename in file_mapping.items():
                try:
                    datasets[table_name] = pd.read_csv(zip_file.open(filename))
                    print(f"✓ Loaded {table_name}: {datasets[table_name].shape[0]:,} rows")
                except KeyError:
                    print(f"⚠ File {filename} not found in zip")
                    continue
        
        return datasets
        
    except Exception as e:
        print(f"Error loading data from zip: {e}")
        print("Creating sample data for demonstration...")
        return create_sample_olist_data()

def create_sample_olist_data():
    """
    Create sample Olist-like data as fallback.
    This mimics the structure and patterns of real Olist data.
    """
    np.random.seed(42)
    
    # Sample customer data
    n_customers = 5000
    brazilian_states = ['SP', 'RJ', 'MG', 'RS', 'PR', 'SC', 'BA', 'GO', 'PE', 'CE']
    
    customers = pd.DataFrame({
        'customer_id': [f'cust_{i:06d}' for i in range(n_customers)],
        'customer_state': np.random.choice(brazilian_states, n_customers, 
                                         p=[0.4, 0.15, 0.12, 0.08, 0.06, 0.04, 0.04, 0.03, 0.03, 0.05]),
        'customer_city': np.random.choice(['São Paulo', 'Rio de Janeiro', 'Belo Horizonte', 
                                         'Porto Alegre', 'Curitiba'], n_customers)
    })
    
    # Sample order data
    n_orders = 8000
    start_date = datetime(2017, 1, 1)
    end_date = datetime(2018, 8, 31)
    
    orders = pd.DataFrame({
        'order_id': [f'ord_{i:08d}' for i in range(n_orders)],
        'customer_id': np.random.choice(customers['customer_id'], n_orders),
        'order_status': np.random.choice(['delivered', 'shipped', 'processing', 'canceled'], 
                                       n_orders, p=[0.85, 0.08, 0.04, 0.03]),
        'order_purchase_timestamp': pd.date_range(start_date, end_date, periods=n_orders),
        'order_delivered_customer_date': pd.date_range(start_date + timedelta(days=5), 
                                                     end_date + timedelta(days=10), periods=n_orders)
    })
    
    # Sample order items with pricing
    n_items = 12000
    order_items = pd.DataFrame({
        'order_id': np.random.choice(orders['order_id'], n_items),
        'product_id': [f'prod_{i:06d}' for i in np.random.randint(0, 2000, n_items)],
        'seller_id': [f'sell_{i:04d}' for i in np.random.randint(0, 500, n_items)],
        'price': np.random.lognormal(3.5, 0.8, n_items),
        'freight_value': np.random.exponential(15, n_items)
    })
    
    # Sample product categories
    categories = ['health_beauty', 'computers_accessories', 'furniture_decor', 
                 'sports_leisure', 'housewares', 'watches_gifts', 'telephony', 
                 'auto', 'toys', 'cool_stuff']
    
    products = pd.DataFrame({
        'product_id': [f'prod_{i:06d}' for i in range(2000)],
        'product_category_name': np.random.choice(categories, 2000),
        'product_weight_g': np.random.lognormal(6, 1, 2000),
        'product_length_cm': np.random.lognormal(3, 0.5, 2000),
        'product_height_cm': np.random.lognormal(2.5, 0.5, 2000),
        'product_width_cm': np.random.lognormal(3, 0.5, 2000)
    })
    
    print("✓ Sample datasets created successfully")
    print(f"  - Customers: {customers.shape[0]:,} rows")
    print(f"  - Orders: {orders.shape[0]:,} rows")
    print(f"  - Order Items: {order_items.shape[0]:,} rows")
    print(f"  - Products: {products.shape[0]:,} rows")
    
    return {
        'customers': customers,
        'orders': orders,
        'order_items': order_items,
        'products': products
    }

# Load the data
print("Loading Olist e-commerce dataset...")
data = load_olist_data_from_zip()
print("\n📊 Data loading complete!")

Loading Olist e-commerce dataset...
Extracting data files...
✓ Loaded customers: 99,441 rows
✓ Loaded orders: 99,441 rows
✓ Loaded order_items: 112,650 rows
✓ Loaded products: 32,951 rows
✓ Loaded sellers: 3,095 rows
✓ Loaded payments: 103,886 rows
✓ Loaded reviews: 99,224 rows
✓ Loaded geolocation: 1,000,163 rows

📊 Data loading complete!


## 2. Data Preparation for Visualization

Before creating interactive visualizations, let's prepare our data by joining the relevant tables and creating business metrics.

In [3]:
# Prepare comprehensive business dataset
def prepare_business_dataset(data):
    """
    Create a comprehensive business dataset by joining multiple tables
    and calculating key business metrics.
    """
    print("Preparing business dataset...")
    
    # Start with orders
    df = data['orders'].copy()
    
    # Convert date columns
    date_columns = ['order_purchase_timestamp', 'order_delivered_customer_date']
    for col in date_columns:
        if col in df.columns:
            df[col] = pd.to_datetime(df[col], errors='coerce')
    
    # Add customer information
    if 'customers' in data:
        df = df.merge(data['customers'], on='customer_id', how='left')
    
    # Add order value information
    if 'order_items' in data:
        order_values = data['order_items'].groupby('order_id').agg({
            'price': 'sum',
            'freight_value': 'sum',
            'product_id': 'count'
        }).reset_index()
        order_values.columns = ['order_id', 'order_value', 'freight_value', 'items_count']
        order_values['total_value'] = order_values['order_value'] + order_values['freight_value']
        
        df = df.merge(order_values, on='order_id', how='left')
    
    # Add product category information
    if 'order_items' in data and 'products' in data:
        # Get primary category for each order (most expensive item)
        order_categories = data['order_items'].merge(
            data['products'][['product_id', 'product_category_name']], 
            on='product_id', how='left'
        )
        
        # Get the category of the most expensive item per order
        primary_categories = order_categories.loc[
            order_categories.groupby('order_id')['price'].idxmax()
        ][['order_id', 'product_category_name']]
        
        df = df.merge(primary_categories, on='order_id', how='left')
    
    # Create time-based features
    if 'order_purchase_timestamp' in df.columns:
        df['year'] = df['order_purchase_timestamp'].dt.year
        df['month'] = df['order_purchase_timestamp'].dt.month
        df['quarter'] = df['order_purchase_timestamp'].dt.quarter
        df['weekday'] = df['order_purchase_timestamp'].dt.day_name()
        df['hour'] = df['order_purchase_timestamp'].dt.hour
    
    # Calculate delivery time (if available)
    if all(col in df.columns for col in ['order_purchase_timestamp', 'order_delivered_customer_date']):
        df['delivery_days'] = (df['order_delivered_customer_date'] - df['order_purchase_timestamp']).dt.days
    
    # Clean up missing values for better visualization
    df = df.fillna({
        'order_value': 0,
        'freight_value': 0,
        'total_value': 0,
        'items_count': 1,
        'customer_state': 'Unknown',
        'customer_city': 'Unknown',
        'product_category_name': 'uncategorized'
    })
    
    print(f"✓ Business dataset prepared: {df.shape[0]:,} orders with {df.shape[1]} features")
    return df

# Prepare the dataset
business_df = prepare_business_dataset(data)

# Display basic information
print("\n📈 Dataset Overview:")
print(f"Date range: {business_df['order_purchase_timestamp'].min()} to {business_df['order_purchase_timestamp'].max()}")
print(f"Total orders: {business_df.shape[0]:,}")
print(f"Total value: ${business_df['total_value'].sum():,.2f}")
print(f"Average order value: ${business_df['total_value'].mean():.2f}")

# Show sample data
print("\n🔍 Sample Business Data:")
display(business_df[['order_id', 'customer_state', 'order_status', 'total_value', 
                    'product_category_name', 'year', 'month']].head())

Preparing business dataset...
✓ Business dataset prepared: 99,441 orders with 23 features

📈 Dataset Overview:
Date range: 2016-09-04 21:15:19 to 2018-10-17 17:30:18
Total orders: 99,441
Total value: $15,843,553.24
Average order value: $159.33

🔍 Sample Business Data:


,order_id,customer_state,order_status,total_value,product_category_name,year,month
0,e481f51cbdc54678b7cc49136f2d6af7,SP,delivered,38.71,utilidades_domesticas,2017,10
1,53cdb2fc8bc7dce0b6741e2150273451,BA,delivered,141.46,perfumaria,2018,7
2,47770eb9100c2d0c44946d9cf07ec65d,GO,delivered,179.12,automotivo,2018,8
3,949d5b44dbf5de918fe9c16f97b45f8a,RN,delivered,72.20,pet_shop,2017,11
4,ad21c59c0840e6cb83a9ceb5573f8159,SP,delivered,28.62,papelaria,2018,2


## 3. Introduction to Plotly Express

Plotly Express is a high-level interface for creating interactive visualizations quickly. It's perfect for business analytics because it automatically adds interactivity features like:

- **Hover information** - Detailed data on mouseover
- **Zoom and pan** - Explore data at different scales
- **Legend interactions** - Click to show/hide series
- **Selection tools** - Select and highlight data points

Let's start with basic interactive charts using our Olist data.

### 3.1 Interactive Scatter Plots - Customer Analysis

In [4]:
# Create an interactive scatter plot to analyze order patterns
fig = px.scatter(
    business_df[business_df['total_value'] > 0].sample(n=min(2000, len(business_df))),  # Sample for performance
    x='items_count',
    y='total_value',
    color='customer_state',
    size='freight_value',
    hover_data=['order_id', 'product_category_name', 'order_status'],
    title='📊 Order Analysis: Items vs Total Value by State',
    labels={
        'items_count': 'Number of Items in Order',
        'total_value': 'Total Order Value ($)',
        'customer_state': 'Customer State',
        'freight_value': 'Shipping Cost ($)'
    },
    color_discrete_sequence=px.colors.qualitative.Set3
)

# Customize the layout
fig.update_layout(
    width=900,
    height=600,
    title_font_size=16,
    showlegend=True,
    legend=dict(orientation="v", yanchor="top", y=1, xanchor="left", x=1.02)
)

# Add professional styling
fig.update_traces(opacity=0.7)

fig.show()

print("💡 Interactive Features to Try:")
print("  • Hover over points to see detailed information")
print("  • Click legend items to show/hide states")
print("  • Use zoom tools to explore dense areas")
print("  • Double-click legend to isolate a single state")

💡 Interactive Features to Try:
  • Hover over points to see detailed information
  • Click legend items to show/hide states
  • Use zoom tools to explore dense areas
  • Double-click legend to isolate a single state


### 3.2 Interactive Time Series - Sales Trends

In [8]:
# Prepare monthly sales data
monthly_sales = business_df.groupby(['year', 'month']).agg({
    'total_value': 'sum',
    'order_id': 'count'
}).reset_index()

monthly_sales['date'] = pd.to_datetime(monthly_sales[['year', 'month']].assign(day=1))
monthly_sales['avg_order_value'] = monthly_sales['total_value'] / monthly_sales['order_id']

# Create interactive time series
fig = px.line(
    monthly_sales,
    x='date',
    y='total_value',
    title='📈 Monthly Sales Revenue Trend',
    labels={
        'date': 'Month',
        'total_value': 'Total Revenue ($)',
        'order_id': 'Number of Orders'
    },
    hover_data=['order_id', 'avg_order_value']
)

# Add markers to the line
fig.update_traces(mode='lines+markers', marker=dict(size=8))

# Customize layout
fig.update_layout(
    width=900,
    height=500,
    title_font_size=16,
    xaxis_title_font_size=14,
    yaxis_title_font_size=14,
    hovermode='x unified'
)

# Format y-axis as currency
# fig.update_yaxis(tickformat='$,.0f')

fig.show()


# Create a secondary chart showing order count
fig2 = px.line(
    monthly_sales,
    x='date',
    y='order_id',
    title='📊 Monthly Order Volume',
    labels={
        'date': 'Month',
        'order_id': 'Number of Orders'
    },
    line_shape='spline'
)

fig2.update_traces(line_color='orange', mode='lines+markers')
fig2.update_layout(width=900, height=400)
fig2.show()

print("\n💡 Time Series Insights:")
print(f"  • Peak month: {monthly_sales.loc[monthly_sales['total_value'].idxmax(), 'date'].strftime('%B %Y')}")
print(f"  • Revenue growth: {((monthly_sales['total_value'].iloc[-1] / monthly_sales['total_value'].iloc[0]) - 1) * 100:.1f}%")
print(f"  • Average monthly orders: {monthly_sales['order_id'].mean():.0f}")


💡 Time Series Insights:
  • Peak month: November 2017
  • Revenue growth: -100.0%
  • Average monthly orders: 3978


### 3.3 Interactive Bar Charts - Category Performance

In [9]:
# Analyze category performance
category_performance = business_df.groupby('product_category_name').agg({
    'total_value': 'sum',
    'order_id': 'count',
    'items_count': 'sum'
}).reset_index()

category_performance['avg_order_value'] = category_performance['total_value'] / category_performance['order_id']
category_performance = category_performance.sort_values('total_value', ascending=False).head(15)

# Create interactive bar chart
fig = px.bar(
    category_performance,
    x='product_category_name',
    y='total_value',
    title='🛍️ Top Product Categories by Revenue',
    labels={
        'product_category_name': 'Product Category',
        'total_value': 'Total Revenue ($)',
        'order_id': 'Number of Orders',
        'avg_order_value': 'Average Order Value ($)'
    },
    hover_data=['order_id', 'avg_order_value'],
    color='avg_order_value',
    color_continuous_scale='Viridis'
)

# Customize layout
fig.update_layout(
    width=1000,
    height=600,
    title_font_size=16,
    xaxis_tickangle=-45,
    xaxis_title_font_size=14,
    yaxis_title_font_size=14
)

# Format y-axis as currency
# fig.update_yaxis(tickformat='$,.0f')

fig.show()

print("\n🏆 Top Performing Categories:")
for i, row in category_performance.head(5).iterrows():
    print(f"  {i+1}. {row['product_category_name']}: ${row['total_value']:,.0f} ({row['order_id']:,} orders)")


🏆 Top Performing Categories:
  12. beleza_saude: $1,441,978 (8,801 orders)
  67. relogios_presentes: $1,306,855 (5,607 orders)
  14. cama_mesa_banho: $1,242,902 (9,328 orders)
  33. esporte_lazer: $1,156,092 (7,683 orders)
  45. informatica_acessorios: $1,059,089 (6,670 orders)


### 3.4 Interactive Geographic Visualization - State Analysis

In [10]:
# Analyze sales by state
state_analysis = business_df.groupby('customer_state').agg({
    'total_value': 'sum',
    'order_id': 'count',
    'freight_value': 'mean'
}).reset_index()

state_analysis['avg_order_value'] = state_analysis['total_value'] / state_analysis['order_id']
state_analysis = state_analysis.sort_values('total_value', ascending=True)

# Create horizontal bar chart for better state name readability
fig = px.bar(
    state_analysis,
    x='total_value',
    y='customer_state',
    orientation='h',
    title='🗺️ Sales Performance by Brazilian State',
    labels={
        'customer_state': 'State',
        'total_value': 'Total Revenue ($)',
        'order_id': 'Number of Orders',
        'avg_order_value': 'Average Order Value ($)',
        'freight_value': 'Average Shipping Cost ($)'
    },
    hover_data=['order_id', 'avg_order_value', 'freight_value'],
    color='avg_order_value',
    color_continuous_scale='RdYlBu_r'
)

# Customize layout
fig.update_layout(
    width=900,
    height=600,
    title_font_size=16,
    xaxis_title_font_size=14,
    yaxis_title_font_size=14
)

# Format x-axis as currency
# fig.update_xaxis(tickformat='$,.0f')

fig.show()

print("\n🌟 Geographic Insights:")
top_state = state_analysis.iloc[-1]
print(f"  • Highest revenue state: {top_state['customer_state']} (${top_state['total_value']:,.0f})")
print(f"  • Most orders: {state_analysis.loc[state_analysis['order_id'].idxmax(), 'customer_state']}")
print(f"  • Highest AOV: {state_analysis.loc[state_analysis['avg_order_value'].idxmax(), 'customer_state']}")


🌟 Geographic Insights:
  • Highest revenue state: SP ($5,921,678)
  • Most orders: SP
  • Highest AOV: PB


## 4. Advanced Plotly Express Features

Now let's explore more advanced features that make Plotly particularly powerful for business analytics.

### 4.1 Animation - Showing Trends Over Time

In [ ]:
# Limit to top 10 states by total revenue for smoother animation
top_10_states = state_analysis.sort_values('total_value', ascending=False).head(10)['customer_state'].tolist()
animation_data = monthly_state_data[monthly_state_data['customer_state'].isin(top_10_states)]

# Create animated scatter plot
fig = px.scatter(
    animation_data,
    x='order_id',
    y='total_value',
    animation_frame='period',
    animation_group='customer_state',
    color='customer_state',
    size='total_value',
    hover_name='customer_state',
    title='🎬 Monthly State Performance Evolution (Top 10 States)',
    labels={
        'order_id': 'Number of Orders',
        'total_value': 'Total Revenue ($)',
        'customer_state': 'State'
    },
    size_max=50
)

fig.update_layout(
    width=900,
    height=600,
    title_font_size=16
)

fig.show()


In [15]:
# Prepare data for animation
monthly_state_data = business_df.groupby(['year', 'month', 'customer_state']).agg({
    'total_value': 'sum',
    'order_id': 'count'
}).reset_index()

monthly_state_data['date'] = pd.to_datetime(monthly_state_data[['year', 'month']].assign(day=1))
monthly_state_data['period'] = monthly_state_data['date'].dt.strftime('%Y-%m')

# Filter to top states for cleaner animation
top_states = state_analysis['customer_state'].tolist()
animation_data = monthly_state_data[monthly_state_data['customer_state'].isin(top_states)]

# Create animated scatter plot
fig = px.scatter(
    animation_data,
    x='order_id',
    y='total_value',
    animation_frame='period',
    animation_group='customer_state',
    color='customer_state',
    size='total_value',
    hover_name='customer_state',
    title='🎬 Monthly State Performance Evolution',
    labels={
        'order_id': 'Number of Orders',
        'total_value': 'Total Revenue ($)',
        'customer_state': 'State'
    },
    size_max=50
)

# Customize animation
fig.update_layout(
    width=900,
    height=600,
    title_font_size=16
)

# Format y-axis as currency
# fig.update_yaxis(tickformat='$,.0f')

fig.show()

print("\n🎬 Animation Controls:")
print("  • Click the play button to see trends over time")
print("  • Use the slider to jump to specific months")
print("  • Pause to examine specific periods")
print("  • Watch how state performance changes month by month")


🎬 Animation Controls:
  • Click the play button to see trends over time
  • Use the slider to jump to specific months
  • Pause to examine specific periods
  • Watch how state performance changes month by month


### 4.2 Facet Plots - Multiple Perspectives

In [16]:
# Create facet plot showing order patterns by day of week for top categories
top_categories = category_performance.head(6)['product_category_name'].tolist()
weekday_data = business_df[
    business_df['product_category_name'].isin(top_categories)
].groupby(['weekday', 'product_category_name']).agg({
    'order_id': 'count',
    'total_value': 'sum'
}).reset_index()

# Define weekday order
weekday_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
weekday_data['weekday'] = pd.Categorical(weekday_data['weekday'], categories=weekday_order, ordered=True)
weekday_data = weekday_data.sort_values('weekday')

# Create facet plot
fig = px.bar(
    weekday_data,
    x='weekday',
    y='order_id',
    facet_col='product_category_name',
    facet_col_wrap=3,
    title='📅 Order Patterns by Day of Week Across Categories',
    labels={
        'weekday': 'Day of Week',
        'order_id': 'Number of Orders',
        'product_category_name': 'Category'
    },
    color='order_id',
    color_continuous_scale='Blues'
)

# Update layout for better readability
fig.update_layout(
    width=1200,
    height=800,
    title_font_size=16,
    showlegend=False
)

# Rotate x-axis labels
fig.update_xaxes(tickangle=45)

fig.show()

print("\n📊 Facet Plot Benefits:")
print("  • Compare patterns across multiple categories simultaneously")
print("  • Identify category-specific trends")
print("  • Spot weekend vs weekday shopping behaviors")
print("  • Each subplot maintains its own scale for better comparison")


📊 Facet Plot Benefits:
  • Compare patterns across multiple categories simultaneously
  • Identify category-specific trends
  • Spot weekend vs weekday shopping behaviors
  • Each subplot maintains its own scale for better comparison


### 4.3 3D Visualization - Multi-Dimensional Analysis

In [17]:
# Prepare data for 3D analysis
sample_data = business_df[
    (business_df['total_value'] > 0) & 
    (business_df['customer_state'].isin(top_states))
].sample(n=min(1000, len(business_df)))

# Create 3D scatter plot
fig = px.scatter_3d(
    sample_data,
    x='items_count',
    y='total_value',
    z='freight_value',
    color='customer_state',
    size='total_value',
    hover_data=['order_id', 'product_category_name'],
    title='🌐 3D Order Analysis: Items, Value, and Shipping',
    labels={
        'items_count': 'Number of Items',
        'total_value': 'Total Value ($)',
        'freight_value': 'Shipping Cost ($)',
        'customer_state': 'State'
    }
)

# Customize 3D layout
fig.update_layout(
    width=900,
    height=700,
    title_font_size=16,
    scene=dict(
        xaxis_title='Number of Items',
        yaxis_title='Total Value ($)',
        zaxis_title='Shipping Cost ($)',
        camera=dict(
            eye=dict(x=1.2, y=1.2, z=1.2)
        )
    )
)

fig.show()

print("\n🌐 3D Visualization Features:")
print("  • Rotate the plot by dragging")
print("  • Zoom in/out with mouse wheel")
print("  • Hover over points for detailed information")
print("  • Explore relationships between three variables simultaneously")
print("  • Identify outliers and patterns in 3D space")


🌐 3D Visualization Features:
  • Rotate the plot by dragging
  • Zoom in/out with mouse wheel
  • Hover over points for detailed information
  • Explore relationships between three variables simultaneously
  • Identify outliers and patterns in 3D space


## 5. Business Intelligence Dashboard Preview

Let's create a mini-dashboard showing key business metrics using Plotly's subplot functionality.

In [18]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# Create subplots for dashboard
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=(
        'Monthly Revenue Trend',
        'Top Categories by Revenue',
        'State Performance',
        'Order Value Distribution'
    ),
    specs=[
        [{"secondary_y": False}, {"secondary_y": False}],
        [{"secondary_y": False}, {"secondary_y": False}]
    ]
)

# Chart 1: Monthly Revenue Trend
fig.add_trace(
    go.Scatter(
        x=monthly_sales['date'],
        y=monthly_sales['total_value'],
        mode='lines+markers',
        name='Revenue',
        line=dict(color='#1f77b4'),
        hovertemplate='Date: %{x}<br>Revenue: $%{y:,.0f}<extra></extra>'
    ),
    row=1, col=1
)

# Chart 2: Top Categories
top_5_categories = category_performance.head(5)
fig.add_trace(
    go.Bar(
        x=top_5_categories['total_value'],
        y=top_5_categories['product_category_name'],
        orientation='h',
        name='Categories',
        marker=dict(color='#ff7f0e'),
        hovertemplate='Category: %{y}<br>Revenue: $%{x:,.0f}<extra></extra>'
    ),
    row=1, col=2
)

# Chart 3: State Performance
top_5_states = state_analysis.tail(5)
fig.add_trace(
    go.Bar(
        x=top_5_states['customer_state'],
        y=top_5_states['total_value'],
        name='States',
        marker=dict(color='#2ca02c'),
        hovertemplate='State: %{x}<br>Revenue: $%{y:,.0f}<extra></extra>'
    ),
    row=2, col=1
)

# Chart 4: Order Value Distribution
fig.add_trace(
    go.Histogram(
        x=business_df[business_df['total_value'] < 500]['total_value'],  # Limit for better visualization
        nbinsx=30,
        name='Order Values',
        marker=dict(color='#d62728'),
        hovertemplate='Value Range: $%{x}<br>Count: %{y}<extra></extra>'
    ),
    row=2, col=2
)

# Update layout
fig.update_layout(
    height=800,
    title_text="📊 Olist E-commerce Business Intelligence Dashboard",
    title_font_size=20,
    showlegend=False
)

# Update axes labels
fig.update_xaxes(title_text="Date", row=1, col=1)
fig.update_yaxes(title_text="Revenue ($)", row=1, col=1, tickformat='$,.0f')

fig.update_xaxes(title_text="Revenue ($)", row=1, col=2, tickformat='$,.0f')
fig.update_yaxes(title_text="Category", row=1, col=2)

fig.update_xaxes(title_text="State", row=2, col=1)
fig.update_yaxes(title_text="Revenue ($)", row=2, col=1, tickformat='$,.0f')

fig.update_xaxes(title_text="Order Value ($)", row=2, col=2)
fig.update_yaxes(title_text="Frequency", row=2, col=2)

fig.show()

print("\n📊 Dashboard Features:")
print("  • Multiple related visualizations in one view")
print("  • Each chart maintains independent interactivity")
print("  • Comprehensive business overview at a glance")
print("  • Professional layout suitable for presentations")


📊 Dashboard Features:
  • Multiple related visualizations in one view
  • Each chart maintains independent interactivity
  • Comprehensive business overview at a glance
  • Professional layout suitable for presentations


## 6. Key Takeaways and Business Impact

### What We've Learned:

1. **Plotly Express Fundamentals**
   - Quick creation of interactive charts
   - Automatic hover information and zooming
   - Color mapping and sizing for multiple dimensions

2. **Business Analytics Applications**
   - Customer behavior analysis through scatter plots
   - Sales trend identification with time series
   - Category performance comparison with bar charts
   - Geographic insights through state analysis

3. **Advanced Interactive Features**
   - Animation for temporal analysis
   - Facet plots for multi-category comparison
   - 3D visualization for complex relationships
   - Dashboard creation with subplots

### Business Value:

- **Enhanced Engagement**: Interactive charts keep stakeholders engaged
- **Self-Service Analytics**: Users can explore data independently
- **Better Decision Making**: Interactive features reveal hidden insights
- **Professional Presentation**: Publication-ready visualizations

### Next Steps:
In the next session, we'll build complete interactive dashboards with advanced layouts and explore how to combine multiple visualizations into comprehensive business intelligence tools.

## 7. Practice Exercise

**Your Turn! 🚀**

Create an interactive visualization that answers this business question:
*"How do shipping costs vary by state and product category, and what impact does this have on customer behavior?"*

**Requirements:**
1. Use the business_df dataset
2. Include at least 3 interactive features
3. Make it visually appealing and business-ready
4. Add meaningful hover information

**Hint:** Consider using scatter plots, color coding, and size mapping to show multiple dimensions simultaneously.

In [ ]:
# Your practice exercise code here
# Create an interactive visualization to analyze shipping costs and customer behavior

# Step 1: Prepare the data
# Step 2: Choose appropriate chart type
# Step 3: Add interactive features
# Step 4: Customize for business presentation

print("🎯 Start your analysis here!")
print("Think about what story you want to tell with the data...")